# Démonstration : Stream Ciphers

**Objectifs** :
- Comprendre la construction PRG → Stream Cipher
- Implémenter ChaCha20 (stream cipher moderne)
- Comparer avec OTP
- Démontrer l'importance du nonce unique

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import secrets
import os

## 1. Rappel : One-Time Pad (OTP)

**Construction** : $c = m \oplus k$ où $k$ est vraiment aléatoire et $|k| = |m|$

**Problème** : Clé aussi longue que le message → impraticable

In [ ]:
# OTP : clé de même longueur que le message
message = b"This is a secret message that requires a very long key!"
key_otp = secrets.token_bytes(len(message))  # Clé aussi longue !

print(f"Message : {len(message)} bytes")
print(f"Clé OTP : {len(key_otp)} bytes")
print(f"\n⚠️ Clé aussi longue que le message = problème pratique !")

## 2. Stream Cipher : Relaxer les contraintes

**Idée** : Utiliser un **Pseudorandom Generator (PRG)** pour étendre une courte clé

**Construction** :
1. Clé courte : $k \in \{0,1\}^\lambda$ (ex: 128 bits)
2. PRG : $G(k) \to \{0,1\}^n$ où $n \gg \lambda$
3. Chiffrement : $c = m \oplus G(k)$

**Compromis** : Sécurité parfaite → Sécurité computationnelle

## 3. ChaCha20 : Stream Cipher Moderne

**Caractéristiques** :
- Clé : 256 bits (32 bytes)
- Nonce : 96 bits (12 bytes)
- Compteur : 32 bits
- Très rapide (logiciel pur, sans accélération matérielle)
- Utilisé dans TLS 1.3, WireGuard VPN, SSH

In [ ]:
def chacha20_encrypt(plaintext: bytes, key: bytes, nonce: bytes) -> bytes:
    """
    Chiffre avec ChaCha20.
    
    Args:
        plaintext: Message à chiffrer
        key: Clé 256 bits (32 bytes)
        nonce: Nonce 96 bits (12 bytes) - DOIT être unique
    
    Returns:
        Ciphertext (même longueur que plaintext)
    """
    algorithm = algorithms.ChaCha20(key, nonce)
    cipher = Cipher(algorithm, mode=None, backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    return ciphertext

def chacha20_decrypt(ciphertext: bytes, key: bytes, nonce: bytes) -> bytes:
    """
    Déchiffre avec ChaCha20 (identique à l'encryption !).
    """
    return chacha20_encrypt(ciphertext, key, nonce)  # XOR est involutif

In [ ]:
# Test ChaCha20
message = b"Secret message encrypted with ChaCha20 stream cipher!"

# Génération clé et nonce
key_chacha = secrets.token_bytes(32)  # 256 bits
nonce = secrets.token_bytes(12)       # 96 bits

print("=" * 70)
print("CHIFFREMENT ChaCha20")
print("=" * 70)
print(f"\nMessage    : {message}")
print(f"Taille     : {len(message)} bytes")
print(f"\nClé (hex)  : {key_chacha.hex()}")
print(f"Taille clé : {len(key_chacha)} bytes = {len(key_chacha)*8} bits")
print(f"\nNonce (hex): {nonce.hex()}")
print(f"Taille     : {len(nonce)} bytes = {len(nonce)*8} bits")

# Chiffrement
ciphertext = chacha20_encrypt(message, key_chacha, nonce)
print(f"\nCiphertext : {ciphertext.hex()}")
print(f"Taille     : {len(ciphertext)} bytes (identique au message !)")

# Déchiffrement
decrypted = chacha20_decrypt(ciphertext, key_chacha, nonce)
print(f"\nDéchiffré  : {decrypted}")

assert decrypted == message
print("\n✅ Chiffrement/Déchiffrement réussi !")

## 4. Comparaison OTP vs ChaCha20

In [ ]:
import pandas as pd

# Taille du message
msg_size = 1024  # 1 KB

comparison = pd.DataFrame({
    'Aspect': [
        'Taille clé',
        'Réutilisation clé',
        'Sécurité',
        'Vitesse',
        'Usage pratique'
    ],
    'OTP': [
        f'{msg_size} bytes (= message)',
        '❌ JAMAIS (catastrophe)',
        '✅ Parfaite (prouvée)',
        '✅ Très rapide (XOR)',
        '❌ Impraticable'
    ],
    'ChaCha20': [
        '32 bytes (fixe)',
        '✅ Oui (avec nonces différents)',
        '⚙️ Computationnelle',
        '✅ Très rapide',
        '✅ Largement utilisé'
    ]
})

print("\n" + "=" * 70)
print("COMPARAISON : OTP vs ChaCha20")
print("=" * 70)
print(f"\nTaille message : {msg_size} bytes\n")
print(comparison.to_string(index=False))

print(f"\n💡 ChaCha20 : {msg_size}/{32} = {msg_size/32:.0f}x plus efficace en taille de clé !")

## 5. Importance du Nonce Unique

**Règle d'or** : JAMAIS réutiliser la paire (clé, nonce) !

**Si violation** : Même attaque que Two-Time Pad

In [ ]:
# Démonstration : Réutilisation de nonce (ERREUR)
print("=" * 70)
print("DANGER : Réutilisation de Nonce")
print("=" * 70)

# Deux messages différents
msg1 = b"ATTACK AT DAWN  "
msg2 = b"RETREAT AT NIGHT"

# Alice réutilise le même nonce (ERREUR FATALE !)
key = secrets.token_bytes(32)
nonce_reused = secrets.token_bytes(12)

c1 = chacha20_encrypt(msg1, key, nonce_reused)
c2 = chacha20_encrypt(msg2, key, nonce_reused)

print(f"\nMessage 1 : {msg1}")
print(f"Message 2 : {msg2}")
print(f"\nNonce réutilisé : {nonce_reused.hex()}")
print(f"\nCiphertext 1 : {c1.hex()}")
print(f"Ciphertext 2 : {c2.hex()}")

# Eve intercepte et calcule c1 ⊕ c2
xor_result = bytes(a ^ b for a, b in zip(c1, c2))

print(f"\n🔓 Eve calcule c1 ⊕ c2 :")
print(f"Résultat : {xor_result}")

# Vérification : c1 ⊕ c2 = m1 ⊕ m2
expected = bytes(a ^ b for a, b in zip(msg1, msg2))
assert xor_result == expected

print(f"\n⚠️  Eve obtient m1 ⊕ m2 = {expected}")
print(f"\n❌ Avec analyse linguistique, Eve peut retrouver m1 et m2 !")
print(f"\n✅ SOLUTION : Toujours utiliser un nonce UNIQUE par message")

## 6. Gestion des Nonces

**Stratégies pour garantir l'unicité** :

In [ ]:
print("STRATÉGIES DE GESTION DES NONCES")
print("=" * 70)

# Stratégie 1 : Nonce aléatoire
print("\n1️⃣ Nonce aléatoire (simple, recommandé)")
nonce1 = secrets.token_bytes(12)
print(f"   Nonce : {nonce1.hex()}")
print(f"   ✅ Probabilité de collision : ~2^-96 (négligeable)")
print(f"   ⚠️  Limite : ~2^48 messages avec même clé (birthday paradox)")

# Stratégie 2 : Compteur
print("\n2️⃣ Compteur (déterministe, efficace)")
counter = 0
nonce2 = counter.to_bytes(12, 'big')
print(f"   Nonce (compteur=0) : {nonce2.hex()}")
counter += 1
nonce2 = counter.to_bytes(12, 'big')
print(f"   Nonce (compteur=1) : {nonce2.hex()}")
print(f"   ✅ Garantie d'unicité si compteur jamais réinitialisé")
print(f"   ⚠️  Attention : synchronisation entre émetteur/récepteur")

# Stratégie 3 : Hybride
print("\n3️⃣ Hybride (timestamp + aléatoire)")
import time
timestamp = int(time.time()).to_bytes(4, 'big')
random_part = secrets.token_bytes(8)
nonce3 = timestamp + random_part
print(f"   Timestamp : {timestamp.hex()} ({int.from_bytes(timestamp, 'big')})")
print(f"   Random    : {random_part.hex()}")
print(f"   Nonce     : {nonce3.hex()}")
print(f"   ✅ Combine avantages des deux approches")

## 7. Performance

In [ ]:
import time

def benchmark_stream_cipher():
    """
    Benchmark ChaCha20 sur différentes tailles.
    """
    sizes = [1024, 10240, 102400, 1024000]  # 1KB, 10KB, 100KB, 1MB
    iterations = 1000
    
    key = secrets.token_bytes(32)
    
    print("\n" + "=" * 70)
    print("BENCHMARK ChaCha20")
    print("=" * 70)
    print(f"Itérations : {iterations}\n")
    
    for size in sizes:
        data = secrets.token_bytes(size)
        
        start = time.perf_counter()
        for i in range(iterations):
            nonce = i.to_bytes(12, 'big')  # Nonce unique par itération
            ct = chacha20_encrypt(data, key, nonce)
            pt = chacha20_decrypt(ct, key, nonce)
        elapsed = time.perf_counter() - start
        
        throughput = (size * iterations * 2) / elapsed / 1_000_000  # MB/s
        
        print(f"Taille : {size:>7} bytes ({size//1024:>4} KB)")
        print(f"  Temps  : {elapsed:>6.3f}s")
        print(f"  Débit  : {throughput:>6.1f} MB/s")
        print()

benchmark_stream_cipher()

## 8. Cas d'usage réels

**ChaCha20 est utilisé dans** :
- **TLS 1.3** : `TLS_CHACHA20_POLY1305_SHA256` (avec AEAD)
- **WireGuard VPN** : Chiffrement rapide et sécurisé
- **SSH** : `chacha20-poly1305@openssh.com`
- **Google QUIC** : Protocole transport
- **Signal Protocol** : Messagerie chiffrée

**Pourquoi ChaCha20 ?**
- Très rapide sans accélération matérielle (mobile, IoT)
- Résistant aux timing attacks
- Pas de tables S-box (constant-time)
- Conçu par Daniel J. Bernstein (cryptographe renommé)

## Conclusion

**Points clés** :
- Stream cipher = PRG + XOR (relaxation du OTP)
- ChaCha20 : clé courte (32 bytes), nonce unique requis
- ❌ Réutilisation de nonce = catastrophe (Two-Time Pad)
- ✅ Gestion nonces : aléatoire, compteur, ou hybride
- Très rapide, largement déployé (TLS, VPN, SSH)

**Limitations** :
- Pas d'intégrité/authenticité (utiliser AEAD : ChaCha20-Poly1305)
- Nonce management critique

**En pratique** : Toujours utiliser **ChaCha20-Poly1305** (AEAD) plutôt que ChaCha20 seul !